# Bloodtype 

## Import dataframe which contains usernames and according phenotype

In [1]:
import pandas as pd

df = pd.read_csv('final_bloodtype_df.csv')
raw_names = df["name"].tolist()
df

,name,has_Bloodtype
0,4130.23andme.2797,1
1,943.23andme.470,1
2,1.23andme.9,1
3,33.23andme.12,1
4,42.23andme.18,1
...,...,...
369,3867.23andme.3643,4
370,6263.23andme.4912,4
371,9822.23andme.8081,4
372,10348.23andme.8605,4


## Change names in column "name"

In [2]:
temp_user = None

processed_names = []

sample_string = "836.23andme.413"

# delete ".23andme" and replace with "_file" and add "user" in front
def clean_name(filename: str):
    garbage = ".23andme."
    temp_string = filename.replace(garbage,"_file")
    return "user"+temp_string
new_names = []
for i in raw_names:
    new_names.append(clean_name(i))

df["name"] = new_names

df


,name,has_Bloodtype
0,user4130_file2797,1
1,user943_file470,1
2,user1_file9,1
3,user33_file12,1
4,user42_file18,1
...,...,...
369,user3867_file3643,4
370,user6263_file4912,4
371,user9822_file8081,4
372,user10348_file8605,4


## Access files in directory
### all files

In [3]:
import os 

directory = "D:/Leah Data/patientFiles"
for filename in os.scandir(directory):
    if filename.is_file():
        print(filename.path)



D:/Leah Data/patientFiles\user10000_file8266_yearofbirth_unknown_sex_unknown.23andme.txt
D:/Leah Data/patientFiles\user10001_file8265_yearofbirth_unknown_sex_unknown.23andme.txt
D:/Leah Data/patientFiles\user10005_file8270_yearofbirth_unknown_sex_unknown.23andme.txt
D:/Leah Data/patientFiles\user10006_file8271_yearofbirth_unknown_sex_unknown.23andme.txt
D:/Leah Data/patientFiles\user10007_file8272_yearofbirth_unknown_sex_unknown.23andme.txt
D:/Leah Data/patientFiles\user10009_file8274_yearofbirth_unknown_sex_unknown.ftdna-illumina.txt
D:/Leah Data/patientFiles\user10012_file8276_yearofbirth_unknown_sex_unknown.23andme.txt
D:/Leah Data/patientFiles\user10014_file8277_yearofbirth_unknown_sex_unknown.ftdna-illumina.txt
D:/Leah Data/patientFiles\user1001_file496_yearofbirth_unknown_sex_unknown.ancestry.txt
D:/Leah Data/patientFiles\user10020_file8283_yearofbirth_unknown_sex_unknown.23andme.txt
D:/Leah Data/patientFiles\user10021_file8284_yearofbirth_unknown_sex_unknown.23andme.txt
D:/Leah 

### files which end with 23andme.txt and 23andme-exome-vcf.txt

In [4]:

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith("23andme.txt") or filename.endswith("23andme-exome-vcf.txt"):
        print(file)

user10000_file8266_yearofbirth_unknown_sex_unknown.23andme.txt
user10001_file8265_yearofbirth_unknown_sex_unknown.23andme.txt
user10005_file8270_yearofbirth_unknown_sex_unknown.23andme.txt
user10006_file8271_yearofbirth_unknown_sex_unknown.23andme.txt
user10007_file8272_yearofbirth_unknown_sex_unknown.23andme.txt
user10012_file8276_yearofbirth_unknown_sex_unknown.23andme.txt
user10020_file8283_yearofbirth_unknown_sex_unknown.23andme.txt
user10021_file8284_yearofbirth_unknown_sex_unknown.23andme.txt
user10021_file8285_yearofbirth_unknown_sex_unknown.23andme.txt
user10026_file8287_yearofbirth_unknown_sex_unknown.23andme.txt
user10028_file8290_yearofbirth_unknown_sex_unknown.23andme.txt
user10029_file8292_yearofbirth_unknown_sex_unknown.23andme.txt
user1002_file497_yearofbirth_unknown_sex_unknown.23andme.txt
user10031_file8294_yearofbirth_unknown_sex_unknown.23andme.txt
user10032_file8297_yearofbirth_unknown_sex_unknown.23andme.txt
user10033_file8485_yearofbirth_1981_sex_XY.23andme.txt
us

### 23andme.txt files from dataframe, patients bloodtype

In [5]:
# store names of 23andme.txt files in list
patient_files_bloodtype = []


for file in os.listdir(directory):
    if file.startswith(tuple(new_names)):
        patient_files_bloodtype.append(file)

## Iterating over files to clear and save as parquet

In [7]:
import os
import pandas as pd
from time import sleep

#check if lines include a Hash
def hasHash(line: str):
    if "#" in line:
        return True
    else:
        return False

#get RSID from textfile
def getRSID(line: str):
        return line[:line.index("\t")]

#get genotype from textfile
def getGenotype(line: str):
    if line[-3] == "\t":
        return line[-2:-1]
    else:
        return line[-3:-1]

#get position from textfile
def getPosition(line: str):
    counter = 0
    for i, char in enumerate(line):
        if char == "\t":
            counter = counter + 1

        if counter == 2:
            x = line[i+1:]
            return x[:x.index("\t")]


file_location = 'D:/Leah Data/patientFiles'



successOpen = 0
failOpen = 0
failOpenArray = []

run = False

if run:

    # iterate through each filename in the list
    for i, fileName in enumerate(patient_files_bloodtype):
        rsid = []
        position = []
        genotype = []

        #### Condition to check if we want to open the file
        print("Trying to open: ", fileName)
        
        # OPEN
        try:
            my_file = open(file_location+'/'+ fileName)
            fileContents = my_file.readlines()
            # HERE GOES THE PROCESSING - FILE IS OPENED
            for line in fileContents:
                if hasHash(line)==False:
                    rsid.append(getRSID(line))
                    genotype.append(getGenotype(line))
                    position.append(getPosition(line))
                    
            # SAVE FILE WITH THE ACC. DATA
            d = {"rsid":rsid, "position":position, "genotype":genotype}
            df = pd.DataFrame(data = d)
            f_name="D:/bloodtype_parquets/"+str(fileName)+".parquet"
            df.to_parquet(f_name)
            # f_name="D:/bloodtype_parquets/"+str(fileName)+".csv"
            # df.to_csv(f_name)
            successOpen+=1
            my_file.close()
            del df
            print(f"succesfully stored {fileName[:10]}")
        except:
            # print(fileName, "ERROR. MOVING ON.")
            failOpen+=1
            failOpenArray.append(fileName)


        # if i == 5:
        #     break

    print(f"Success opens: {successOpen}\nFailed opens: {failOpen}")

    print("File names of all failed files:")
    print(failOpenArray)


Trying to open:  user10020_file8283_yearofbirth_unknown_sex_unknown.23andme.txt
succesfully stored user10020_
Trying to open:  user10092_file8351_yearofbirth_unknown_sex_unknown.23andme.txt
succesfully stored user10092_
Trying to open:  user1010_file502_yearofbirth_1984_sex_XY.23andme.txt
succesfully stored user1010_f
Trying to open:  user10123_file8395_yearofbirth_2001_sex_XX.23andme.txt
succesfully stored user10123_
Trying to open:  user1020_file507_yearofbirth_unknown_sex_unknown.23andme.txt
succesfully stored user1020_f
Trying to open:  user10216_file8481_yearofbirth_1990_sex_unknown.23andme.txt
succesfully stored user10216_
Trying to open:  user1024_file510_yearofbirth_1971_sex_XX.23andme.txt
succesfully stored user1024_f
Trying to open:  user1028_file514_yearofbirth_unknown_sex_unknown.23andme.txt
succesfully stored user1028_f
Trying to open:  user10302_file8558_yearofbirth_unknown_sex_unknown.23andme.txt
succesfully stored user10302_
Trying to open:  user1033_file567_yearofbirth

## Data Cleaning

In [8]:
import pandas as pd

#Tracker to track RSIDs
RSID_Tracker = {}

#Condition if rsid starts with "r"
def rsidIsAllowed(rsid: str):
	if  rsid[0]=="r":
		return True
	else:
		return False

#Condition if genotype has an empty value or is smaller than 2 letters
def genotypeIsAllowed(genomeValue: str):
	if genomeValue == "--" or len(genomeValue)<2:
		return False
	else:
		return True

import os


read = False

clean_file_direction = "D:/bloodtype_parquets/"


if read:
	common_rsid_counter = 0

	# Iterate through each parquet file and open 
	for i, filename in enumerate(os.scandir("D:/bloodtype_parquets")):		
		# if i == 5:
		# 	break

		# Read parquet file
		print(filename, "opened! Operating now...")
		patient = pd.read_parquet(filename)

		# Initiate new column fields for new "updated" df
		newRSID = []
		newGenotype=[]
		newPos = []

		# 1. get a list from genotype column 
		OG_genotype = patient["genotype"].tolist()
		
		# # 2 get a list from rsid column
		OG_RSID = patient["rsid"].tolist()

		# # 2.5 get list from positions
		OG_pos = patient["position"].tolist()

		print(f"found {len(OG_genotype)} genotypes")

		# 3. for each value in the list, check if genotype is allowed + for each value in the list, check if ID is allowed
		for j, genome in enumerate(OG_genotype):

			if genotypeIsAllowed(genome) and rsidIsAllowed(OG_RSID[j]):
				newGenotype.append(genome)
				newPos.append(OG_pos[j])
				newRSID.append(OG_RSID[j])
			
			# IN CASE OF ERROR TRACKING UNCOMMENT THE CODE BELOW
			
			# elif not rsidIsAllowed(OG_RSID[j]):
			# 	print("NOT ACCEPTING:", OG_RSID[j],". SKIPPING", OG_RSID[j])
			# elif not genotypeIsAllowed(genome):
			# 	print("NOT ACCEPTING:", genome,". SKIPPING",OG_RSID[j])

		# STORE THE RESULTS AS PARQUET FILES
		new_data = {"rsid":newRSID, "position": newPos, "genotype": newGenotype }
		df_new_patient = pd.DataFrame(data = new_data)

		df_new_patient.to_parquet(clean_file_direction + filename.name )
		print("Stored!")

<DirEntry 'user10020_file8283_yearofbirth_unknown_sex_unknown.23andme.txt.parquet'> opened! Operating now...
found 631955 genotypes
Stored!
<DirEntry 'user10092_file8351_yearofbirth_unknown_sex_unknown.23andme.txt.parquet'> opened! Operating now...
found 601881 genotypes
Stored!
<DirEntry 'user1010_file502_yearofbirth_1984_sex_XY.23andme.txt.parquet'> opened! Operating now...
found 574514 genotypes
Stored!
<DirEntry 'user10123_file8395_yearofbirth_2001_sex_XX.23andme.txt.parquet'> opened! Operating now...
found 631955 genotypes
Stored!
<DirEntry 'user1020_file507_yearofbirth_unknown_sex_unknown.23andme.txt.parquet'> opened! Operating now...
found 960613 genotypes
Stored!
<DirEntry 'user10216_file8481_yearofbirth_1990_sex_unknown.23andme.txt.parquet'> opened! Operating now...
found 610545 genotypes
Stored!
<DirEntry 'user1024_file510_yearofbirth_1971_sex_XX.23andme.txt.parquet'> opened! Operating now...
found 960613 genotypes
Stored!
<DirEntry 'user1028_file514_yearofbirth_unknown_sex_u

## Count all RSIDs and unique RSIDs

In [9]:
allRSIDS = []
# allUniqueRSIDS = []

clean_file_direction = "D:/bloodtype_parquets/"
read = False 

if read:

    # Iterate through parquet files
    for i, filename in enumerate(os.scandir(clean_file_direction)):
        print(filename , 'OPEN')
        file = pd.read_parquet(filename)

        # add every single rsid to list
        patientRSIDS = file["rsid"].tolist()
    
        for j, snp in enumerate(patientRSIDS):
            allRSIDS.append(snp)
        
        # if snp not in allUniqueRSIDS:
        #     allUniqueRSIDS.append(snp)

        # allRSIDS.append(patientRSIDS)
        print("Done. Next:")


    try:
        print(len(allRSIDS))
    except:
        print("Couldn't print LEN for some reason")

<DirEntry 'user10020_file8283_yearofbirth_unknown_sex_unknown.23andme.txt.parquet'> OPEN
Done. Next:
<DirEntry 'user10092_file8351_yearofbirth_unknown_sex_unknown.23andme.txt.parquet'> OPEN
Done. Next:
<DirEntry 'user1010_file502_yearofbirth_1984_sex_XY.23andme.txt.parquet'> OPEN
Done. Next:
<DirEntry 'user10123_file8395_yearofbirth_2001_sex_XX.23andme.txt.parquet'> OPEN
Done. Next:
<DirEntry 'user1020_file507_yearofbirth_unknown_sex_unknown.23andme.txt.parquet'> OPEN
Done. Next:
<DirEntry 'user10216_file8481_yearofbirth_1990_sex_unknown.23andme.txt.parquet'> OPEN
Done. Next:
<DirEntry 'user1024_file510_yearofbirth_1971_sex_XX.23andme.txt.parquet'> OPEN
Done. Next:
<DirEntry 'user1028_file514_yearofbirth_unknown_sex_unknown.23andme.txt.parquet'> OPEN
Done. Next:
<DirEntry 'user10302_file8558_yearofbirth_unknown_sex_unknown.23andme.txt.parquet'> OPEN
Done. Next:
<DirEntry 'user1033_file567_yearofbirth_1992_sex_XY.23andme.txt.parquet'> OPEN
Done. Next:
<DirEntry 'user10348_file8605_yearo

In [10]:
# Create .csv file with all listed RSIDS 

import pandas as pd 
#r = pd.DataFrame({"all_rsids":allRSIDS})
#r

#r.to_csv("ALL_RSIDS_Bloodtype.csv", index=False)

## List rsids in patients and count them

In [11]:
RSID_Tracker = {}

frame = pd.read_csv("ALL_RSIDS_Bloodtype.csv")

print("Opening done.")
print("Counting now.")

read = False
if read:

    # Iterate through each RSID in list and count how often they occure
    for i, rsid in enumerate(frame["all_rsids"].tolist()):
    
    
        if rsid in RSID_Tracker:
            RSID_Tracker[rsid]+=1
        else:
            RSID_Tracker[rsid] = 1

    import json

    # Save results as jason file
    print("Done. Attempting to save as JSON...")
    with open('counted_RSIDS_bloodtype.json', 'w') as fp:
        json.dump(RSID_Tracker, fp)

    print("Done!")

Opening done.
Counting now.
Done. Attempting to save as JSON...
Done!


## rsids which occure at least in 95,9% of patients

In [19]:
import json
import pandas as pd

# Calculation of in how many patients a rsid needs to occure to show a minimun occurence og 97% 
n_minimum_occurence = int((346/100)* 95.95)+1
print(n_minimum_occurence)

# Open file
print("Opening JSON FILE")

f = open("counted_RSIDS_bloodtype.json")
data = json.load(f)

print(len(data))

# Create list with common rsids
most_common_rsids_bloodtype_patients = []

for i, key in enumerate(data):
    # print(i)
    if data[key] >= n_minimum_occurence:

        if key not in most_common_rsids_bloodtype_patients:
            most_common_rsids_bloodtype_patients.append(key)

f.close()
print(len(most_common_rsids_bloodtype_patients))

store = False

# Create .csv file to store the most common rsids
if store:
    print("Done. Attempting to store....")
    common_rsids = pd.DataFrame({"common_rsids":most_common_rsids_bloodtype_patients})
    common_rsids.to_csv("common_rsids_bloodtype.csv", index=False)
    print("Done storing.")

332
Opening JSON FILE
2402229
37359
Done. Attempting to store....
Done storing.


## Get username of patients with most common rsids

In [20]:
c = pd.read_csv("common_rsids_bloodtype.csv")

# Create a list of common rsids and look at the lenght of the list
commons = c["common_rsids"].tolist()
len(commons)

user_with_common_rsids = []

user_names = []

run = True

if run:

    # # If 100% of rsid are in file
    def isFit(patient_rsid_list: list, necessary: list):
        
        is_included = 0
        
        for k, value in enumerate(necessary):
            if value in patient_rsid_list:
                is_included+=1
        
        if is_included/len(necessary)>=1:
            return True
        else:
            return False

    clean_file_direction = "D:/bloodtype_parquets/"

    for j, filename in enumerate(os.scandir(clean_file_direction)):
        
        # Open file and read 
        print(j, filename, 'OPEN')
        file = pd.read_parquet(filename)

        if isFit(file["rsid"].tolist(),commons):
            print(filename,"is fit")
            user_names.append(filename.name)
        else:
            print(filename," NOT FIT !!!!!!")

    print(user_names)

    users = pd.DataFrame({"userID":user_names})

    users.to_csv("37k_Bloodtype_Users_Cleaned.csv", index=False)

    

0 <DirEntry 'user10020_file8283_yearofbirth_unknown_sex_unknown.23andme.txt.parquet'> OPEN
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Lea\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Lea\AppData\Local\Temp\ipykernel_20736\3884305391.py", line 37, in <module>
    if isFit(file["rsid"].tolist(),commons):
  File "C:\Users\Lea\AppData\Local\Temp\ipykernel_20736\3884305391.py", line -1, in isFit
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Lea\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\interactiveshell.py", line 2052, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C:\Users\Lea\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\Loc